In [34]:
!pip install  pandarallel mitosheet --quiet


In [26]:
import pandas as pd
from collections import Counter
import itertools
import mitosheet
from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords


In [3]:

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [25]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [27]:
stemmer = PorterStemmer()
stops = set(stopwords.words('english'))


In [4]:
df = pd.read_parquet("../data/cleaned_input.parquet")

In [5]:
df_product = df[['product_id','product_title','url_product','url_image']].drop_duplicates(['product_id']).reset_index(drop=True)
df_product

,product_id,product_title,url_product,url_image
0,B000MOO21W,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceiling Mounted Fan,https://www.amazon.com/dp/B000MOO21W,http://images.amazon.com/images/P/B000MOO21W.01._SCLZZZZZZZ_.jpg
1,B07X3Y6B1V,"Homewerks 7141-80 Bathroom Fan Integrated LED Light Ceiling Mount Exhaust Ventilation, 1.1 Sones, 80 CFM",https://www.amazon.com/dp/B07X3Y6B1V,http://images.amazon.com/images/P/B07X3Y6B1V.01._SCLZZZZZZZ_.jpg
2,B07WDM7MQQ,"Homewerks 7140-80 Bathroom Fan Ceiling Mount Exhaust Ventilation, 1.5 Sones, 80 CFM, White",https://www.amazon.com/dp/B07WDM7MQQ,http://images.amazon.com/images/P/B07WDM7MQQ.01._SCLZZZZZZZ_.jpg
3,B07RH6Z8KW,Delta Electronics RAD80L BreezRadiance 80 CFM Heater/Fan/Light Combo White (Renewed),https://www.amazon.com/dp/B07RH6Z8KW,http://images.amazon.com/images/P/B07RH6Z8KW.01._SCLZZZZZZZ_.jpg
4,B07QJ7WYFQ,Panasonic FV-08VRE2 Ventilation Fan with Recessed LED (Renewed),https://www.amazon.com/dp/B07QJ7WYFQ,http://images.amazon.com/images/P/B07QJ7WYFQ.01._SCLZZZZZZZ_.jpg
...,...,...,...,...
1207284,0323377092,Neurology Self-Assessment: A Companion to Bradley's Neurology in Clinical Practice,https://www.amazon.com/dp/0323377092,http://images.amazon.com/images/P/0323377092.01._SCLZZZZZZZ_.jpg
1207285,0323287832,"Bradley's Neurology in Clinical Practice, 2-Volume Set",https://www.amazon.com/dp/0323287832,http://images.amazon.com/images/P/0323287832.01._SCLZZZZZZZ_.jpg
1207286,0198566344,Autonomic Failure: A Textbook of Clinical Disorders of the Autonomic Nervous System,https://www.amazon.com/dp/0198566344,http://images.amazon.com/images/P/0198566344.01._SCLZZZZZZZ_.jpg
1207287,0071845836,The Hospital Neurology Book,https://www.amazon.com/dp/0071845836,http://images.amazon.com/images/P/0071845836.01._SCLZZZZZZZ_.jpg


In [6]:
df.shape

(1790990, 11)

In [7]:
df

,query,product_id,relevance_label,product_title,product_description,product_bullet_point,product_brand,product_color,url_product,url_image,query_type
0,revent 80 cfm,B000MOO21W,Irrelevant,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceiling Mounted Fan,None,"WhisperCeiling fans feature a totally enclosed condenser motor and a double-tapered, dolphin-shaped bladed blower wheel to quietly move air\nDesigned to give you continuous, trouble-free operation for many years thanks in part to its high-quality components and permanently lubricated motors which wear at a slower pace\nDetachable adaptors, firmly secured duct ends, adjustable mounting brackets (up to 26-in), fan/motor units that detach easily from the housing and uncomplicated wiring all lend themselves to user-friendly installation\nThis Panasonic fan has a built-in damper to prevent backdraft, which helps to prevent outside air from coming through the fan\n0.35 amp",Panasonic,White,https://www.amazon.com/dp/B000MOO21W,http://images.amazon.com/images/P/B000MOO21W.01._SCLZZZZZZZ_.jpg,other
1,revent 80 cfm,B07X3Y6B1V,Exact,"Homewerks 7141-80 Bathroom Fan Integrated LED Light Ceiling Mount Exhaust Ventilation, 1.1 Sones, 80 CFM",None,"OUTSTANDING PERFORMANCE: This Homewerk's bath fan ensures comfort in your home by quietly eliminating moisture and humidity in the bathroom. This exhaust fan is 1.1 sones at 80 CFM which means it’s able to manage spaces up to 80 square feet and is very quiet..\nBATH FANS HELPS REMOVE HARSH ODOR: When cleaning the bathroom or toilet, harsh chemicals are used and they can leave an obnoxious odor behind. Homewerk’s bathroom fans can help remove this odor with its powerful ventilation\nBUILD QUALITY: Designed to be corrosion resistant with its galvanized steel construction featuring a modern style round shape and has an 4000K Cool White Light LED Light. AC motor.\nEASY INSTALLATION: This exhaust bath fan is easy to install with its no-cut design and ceiling mount ventilation. Ceiling Opening (L) 7-1/2 in x Ceiling Opening (W) 7-1/4 x Ceiling Opening (H) 5-3/4 in. 13 in round grill and 4 in round duct connector.\nHOMEWERKS TRUSTED QUALITY: Be confident in the quality and construction of each and every one of our products. We ensure that all of our products are produced and certified to regional, national and international industry standards. We are proud of the products we sell, you will be too. 3 Year Limited",Homewerks,80 CFM,https://www.amazon.com/dp/B07X3Y6B1V,http://images.amazon.com/images/P/B07X3Y6B1V.01._SCLZZZZZZZ_.jpg,other
2,revent 80 cfm,B07WDM7MQQ,Exact,"Homewerks 7140-80 Bathroom Fan Ceiling Mount Exhaust Ventilation, 1.5 Sones, 80 CFM, White",None,"OUTSTANDING PERFORMANCE: This Homewerk's bath fan ensures comfort in your home by quietly eliminating moisture and humidity in the bathroom. This exhaust fan is 1. 5 sone at 110 CFM which means it’s able to manage spaces up to 110 square feet\nBATH FANS HELPS REMOVE HARSH ODOR: When cleaning the bathroom or toilet, harsh chemicals are used and they can leave an obnoxious odor behind. Homewerk’s bathroom fans can help remove this odor with its powerful ventilation\nBUILD QUALITY: Designed to be corrosion resistant with its galvanized steel construction featuring a grille modern style.\nEASY INSTALLATION: This exhaust bath fan is easy to install with its no-cut design and ceiling mount ventilation. Ceiling Opening (L) 7-1/2 in x Ceiling Opening (W) 7-1/4 x Ceiling Opening (H) 5-3/4 in and a 4"" round duct connector.\nHOMEWERKS TRUSTED QUALITY: Be confident in the quality and construction of each and every one of our products. We ensure that all of our products are produced and certified to regional, national and international industry standards. We are proud of the products we sell, you will be too. 3 Year Limited",Homewerks,White,https://www.amazon.com/dp/B07WDM7MQQ,http://images.amazon.com/images/P/B07WDM7MQQ.01._SCLZZZZZZZ_.jpg,other
3,revent 80 cfm,B07RH6Z8KW,Exact,Delta Electronics RAD80L B

In [28]:
def normalize_and_stem(query):
    words = nltk.word_tokenize(query.lower())
    return [stemmer.stem(word) for word in words]
    
def query_title_similarity(query,title):
    score = 0
    
    query = normalize_and_stem(query)
    title = normalize_and_stem(title)
    
    title_words = set(title)
    query_words = set(query)
    
    for word in query_words:
        if word not in stops and word not in title:
            score+=1
            
    return score

In [8]:
df_res = df [ df['relevance_label']=="Exact" ]

df_res = df_res.groupby(['product_id'],as_index=False).agg(queries=('query',pd.Series.unique), num_queries=('query',len ), title=('product_title', "first") )

df_res = df_res [ df_res['num_queries'] >= 2 ]

In [9]:
df_res

,product_id,queries,num_queries,title
17,0007823509,"[dr seuss book, dr suess books for babies]",2,The Wonderful World of Dr Seuss
18,0007833571,"[tom clancy without remorse book, tom clancys without remorse]",2,WITHOUT REMORSE.
21,0007922736,"[dr seuss book, dr suess books for babies]",2,The Wonderful World of Dr. Seuss 20 Reading Books Collection Gift Box Set
29,0008209103,"[a game of thrones, game if thrones books]",2,GAME OF THRONES_20TH ANNIVE_HB
32,0008249563,"[brave not perfect, brave not perfect by reshma saujani]",2,Brave Not Perfect
...,...,...,...,...
897240,B09J5381TS,"[insulated tumblers for women, wine tumbler not a day over]",2,Gifts for Women 12oz Insulated Wine Tumbler Stemless Insulated Wine Glass with Spill-proof Lid and Two Stainless Steel Straw
897242,B09J53LD37,"[insulated tumblers for women, wine tumbler not a day over]",2,Gifts for Women 12oz Insulated Wine Tumbler Stemless Insulated Wine Glass with Spill-proof Lid and Two Stainless Steel Straw
897265,B09J8SXCV7,"[hoodies black and white, hoodie white and black]",2,Black Hoodie Women Oversized Sweatshirt Cropped Comfy Quarter Zip Pullover Fleece Coats Winter White
897268,B09J94LJ4C,"[baby side sleeping positioner, infant side sleeper wedge]",2,"LVZHIS Baby Adjustable Anti-Rolling Pillow, 45° Inclined Baby Side Pillow, Wedge-Shaped Newborn Anti-Flat Head Pillow, Suitable for 0-6 Months Old Baby"


In [10]:
len(df_res)

166689

In [11]:
(df_res['num_queries'] >= 2).value_counts()

True    166689
Name: num_queries, dtype: int64

In [12]:
(df_res['num_queries'] >= 3).value_counts()

False    100795
True      65894
Name: num_queries, dtype: int64

In [13]:
(df_res['num_queries'] >= 4).value_counts()

False    132735
True      33954
Name: num_queries, dtype: int64

In [14]:
(df_res['num_queries'] >= 5).value_counts()

False    146684
True      20005
Name: num_queries, dtype: int64

In [15]:
df_res [ df_res['num_queries'] > 3 ]

,product_id,queries,num_queries,title
88,0060256656,"[children books by age 1-3, best childrens books by age 1-3, children's books ages 3-5, best books for kids]",4,The Giving Tree
89,0060256672,"[best books for kids, rosie revere engineer, classic poetry books, sidewalk]",4,Where the Sidewalk Ends: Poems and Drawings
175,0060935464,"[book of lists 2, give thanks to the lord, pbs great american read, ro kill a mockingbird, to kill a mockingbird book, to kill a mockingbird graphic novel]",6,To Kill a Mockingbird
266,0061992275,"[books for 4th grade boys, best books for kids, tuesday mooney talks to ghosts, award winning childrens books ages 9-12]",4,The One and Only Ivan
346,0062294180,"[beginner reading book, books for beginning readers girls, level 1 reading books for children, i can read books level 1, i can read]",5,Splat the Cat: I Scream for Ice Cream (I Can Read Level 1)
...,...,...,...,...
895429,B09DY11PCN,"[car carriers rooftop without rack, cargo carrier without roof rack, roof bag for car without rack, rooftop bag without rack]",4,"Rooftop Cargo Carrier, 21 Cubic Feet Rooftop Cargo bag, Waterproof luggage carrier cargo bag, suitable for all cars with/without rack-700D PVC, Includes Non-Slip Mat, 10 Reinforced Straps, 6 Door Hook"
895877,B09FJH1KM5,"[magnetic eyelash kit without eyeliner, magnetic eyelashes without eyeliner or glue, magnetic eyelashes without liner, double magnetic eyelashes without eyeliner]",4,"Magnetic Eyelashes Kit, Glue-free Magnetic Eyelash Clip & Eyelashes Set with Soft Magnetic False Eyelashes, Natural Looking"
896038,B09FPG6RT4,"[laser ingravor, cutter, co2 laser cutter, laser cutting machine, k40 laser]",4,"OMTech 40W Laser Engraver and Cutter, K40 Desktop Engraver with 8x12 Bed, CO2 Laser Engraving and Cutting Machine with Digital Controls Red Dot Pointer Wheels for DIY Woodwork and More, DF0812-40RW"
896201,B09FXQ72Z1,"[magnetic lashes without liner, magnetic eyelash kit without eyeliner, magnetic eyelashes without eyeliner or glue, magnetic eyelashes without liner, double magnetic eyelashes without eyeliner]",5,"Magnetic Eyelashes Applicator Tool Kit,False Eyelashes without Eyeliner, Magnetic Eyelashes no Glue Needed,Eyelashes Set with 2 Pairs Magnetic False Eyelashes,Natural Look Bushy Look"


In [16]:
df_res [ df_res['num_queries'] > 5 ].iloc[78:85]

,product_id,queries,num_queries,title
15659,1524762156,"[low calorie cookbooks, skinnytaste air fryer, skinny taste cookbooks, skinnytaste cookbooks, weight watchers cookbook, the best slow cooker cookbook ever]",6,"Skinnytaste One and Done: 140 No-Fuss Dinners for Your Instant Pot®, Slow Cooker, Air Fryer, Sheet Pan, Skillet, Dutch Oven, and More: A Cookbook"
15687,1524787183,"[book pack for 3 year olds, kids picture books, books for toddlers, kids books, boy toys 1-2 years old, kids camping stuff]",6,Llama Llama Loves Camping
16012,1537010344,"[books for 7 year old, court popular library, books for 4th grade boys, books for boys, books for boys age 8, childrens books by age 6 8 african american, biography on stephen curry]",7,Stephen Curry: The Children's Book: The Boy Who Never Gave Up
16982,1580054846,"[books for babies newborn to one year, new mom gift basket, funny baby gifts for parents, baby gifts for newborns, new mommy, first time mom gift]",6,"The Sh!t No One Tells You: A Guide to Surviving Your Baby's First Year (Sh!t No One Tells You, 1)"
17221,1589255518,"[children books by age 1-3, children’s board books, books for babies newborn to one year, best childrens books by age 1-3, baby books, i love you through and through board book]",6,I Love You to the Moon and Back
18675,1622039076,"[if its not one thing its your mother, its ok not to be ok, its ok that youre not ok, ok to not be ok, how to not die alone, megan devine its ok that youre not ok]",6,It's OK That You're Not OK: Meeting Grief and Loss in a Culture That Doesn't Understand
18761,1623363586,"[vegan diet book, cookbook, goodful cookbook, charred food, soul food cookbook for beginners, healthy eating cookbooks best sellerschristoan, pescatarian cookbooks]",7,Thug Kitchen: The Official Cookbook: Eat Like You Give a F*ck (Thug Kitchen Cookbooks)


In [17]:
df_res [ df_res['num_queries'] > 5 ].iloc[100:200]

,product_id,queries,num_queries,title
23044,193981443X,"[building writers learning without tears, can do cursive handwriting without tears, flip crayons handwriting without tears, handwriting without tears letters and numbers for me, learning without tears, mat man handwriting without tears set, cursive kickoff handwriting without tears, handwriting without tears cursive, handwriting without tears pre-k, spelling without tears, writing without tears kindergarten kit]",11,"Learning Without Tears - Kick Start Kindergarten Student Workbook, Current Edition - Handwriting Without Tears Series - Pre-K Writing Book - Capital and Lowercase Letters - for School or Home Use"
23045,1939814448,"[building writers learning without tears, flip crayons handwriting without tears, handwriting without tears letters and numbers for me, learning without tears, magic without tears, handwriting without tears cursive, handwriting without tears pre-k, learning without tears writing journal, spelling without tears, writing without tears kindergarten kit]",10,"Learning Without Tears - Letters and Numbers for Me Student Workbook, Current Edition - Handwriting Without Tears Series - Kindergarten Writing Book - Capital Letters, Numbers - For School or Home Use"
23046,1939814456,"[building writers learning without tears, can do cursive handwriting without tears, handwriting without tears letters and numbers for me, learning without tears, my printing book handwriting without tears, handwriting without tears cursive, handwriting without tears pre-k, homeschool curriculum 1st grade, learning without tears writing journal, spelling without tears, writing without tears kindergarten kit]",11,"Learning Without Tears - My Printing Book Student Workbook, Current Edition - Handwriting Without Tears Series - 1st Grade Writing Book - Letters, Language Arts Lessons - for School or Home Use"
23047,1939814464,"[handwriting without tears letters and numbers for me, learning without tears, cursive handwriting workbook handwriting without tears, handwriting without tears cursive, handwriting without tears pre-k, learning without tears writing journal, spelling without tears]",7,"Learning Without Tears - Printing Power Student Workbook, Current Edition - Handwriting Without Tears Series - 2nd Grade Writing Book - Writing and Language Arts Lessons - for School or Home Use"
23048,1939814480,"[can do cursive handwriting without tears, flip crayons handwriting without tears, learning without tears, cursive handwriting workbook handwriting without tears, cursive kickoff handwriting without tears, handwriting without tears cursive, handwriting without tears grade 3, handwriting without tears pre-k, learning without tears writing journal, spelling without tears, cursive handwriting workbook, handwriting workbook]",12,"Learning Without Tears - Cursive Handwriting Student Workbook, Current Edition - Handwriting Without Tears Series - 3rd Grade Writing Book - Writing, Language Arts Lessons - for School or Home Use"
...,...,...,...,...
30666,B0002AR0I8,"[dog toy, dog toy anxiety, dog toys, toys for dogs, kong toys for large dogs aggressive chewers, kong dog toys for aggressive chewers, kong wobbler medium, kong dog toys, dog kong, kong activity ball]",10,"KONG - Classic Dog Toy, Durable Natural Rubber- Fun to Chew, Chase and Fetch - for Large Dogs"
30669,B0002AR182,"[teething puppy chew toys, kong dog toys, dog kong, kong, kong dog tug toy, kong activity ball]",6,"KONG - Puppy Toy Natural Teething Rubber - Fun to Chew, Chase and Fetch (Color May Vary) - for Medium Puppies"
30718,B0002ASDIY,"[betta plant bowl, aquarium decorations, aquarium plants, fish tank decorations, aquarium decor, fish tank with plant]",6,"Marina Ecoscaper, Hydrocotyle, Silk Plant Fish Tank Decoration, 8-inch, PP187A1"
31161,B0002H3S5K,"[storage pet, 40 lb dog food storage container, air tight containers for dog food, dog food bin, dry food container for pets, pet food container, dog food storage container]",7,"Ga

In [40]:
df_res = df [ df['relevance_label']=="Exact" ]

df_res['new_tokens'] = df_res.parallel_apply(lambda x: query_title_similarity(x['query'] , x['product_title']), axis=1)

df_res  =df_res [ df_res['new_tokens']>2 ]

df_res = df_res.groupby(['product_id'],as_index=False).agg(queries=('query',pd.Series.unique), num_queries=('query',len ), title=('product_title', "first") )
df_res = df_res [ df_res['num_queries'] >= 2 ]

/tmp/ipykernel_443/459077262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['new_tokens'] = df_res.parallel_apply(lambda x: query_title_similarity(x['query'] , x['product_title']), axis=1)


In [46]:
df_res.head(50)

,product_id,queries,num_queries,title
16,0060245867,"[best childrens books by age 1-3, if you give a pig the white house]",2,If You Give a Mouse a Cookie
18,0060256656,"[children books by age 1-3, best childrens books by age 1-3, children's books ages 3-5, best books for kids]",4,The Giving Tree
19,0060256672,"[best books for kids, rosie revere engineer, classic poetry books]",3,Where the Sidewalk Ends: Poems and Drawings
23,0060555661,"[forex trading for dummies, the interpretation of financial statements benjamin graham]",2,The Intelligent Investor Rev Ed.: The Definitive Book on Value Investing
35,0060935464,"[book of lists 2, give thanks to the lord, pbs great american read]",3,To Kill a Mockingbird
49,0061992275,"[books for 4th grade boys, best books for kids, tuesday mooney talks to ghosts, award winning childrens books ages 9-12]",4,The One and Only Ivan
53,0062060627,"[all the light we can not see paperback, trojans fan store]",2,The Song of Achilles: A Novel
77,0062304240,"[level 1 reading books for children, i can read books level 1]",2,Pete the Cat's Super Cool Reading Collection: 5 I Can Read Favorites! (My First I Can Read)
80,0062334204,"[books for 7 year old, 6 grade reading books sets]",2,Amelia Bedelia Chapter Book 4-Book Box Set: Books 1-4
83,0062352156,"[beginning reader books, books for beginning readers girls]",2,Pinkalicious 12-Book Phonics Fun!: Includes 12 Mini-Books Featuring Short and Long Vowel Sounds (My First I Can Read)


In [47]:
df_res.tail(50)

,product_id,queries,num_queries,title
108416,B093KHND4N,"[i tested positive but not for covid, tested positive but not for covid]",2,"YouNique Designs Pregnant Coffee Mug, 11 Ounces, Pregnant Wife Mug, Pregnancy Announcement Mug for Dad and Grandparents, Mommy To Be Cup, Expecting Mom Coffee Mug (Black Handle)"
108473,B093TGWZHC,"[kids vitamins without gelatin, vitamins for toddlers age 2]",2,"Centrum Kids' Organic Multigummies, Kids Multivitamin Gummies, Organic Multivitamin for Kids with Essential Nutrients for Immune Support, Muscle Function, and Brain Health - 90 Count"
108489,B093X45CDR,"[hair dye for dark hair without bleach, purple hair dye for dark hair without bleach]",2,"Garnier Nutrisse Ultra Nourishing Hair Color Creme with Triple Oils, Permanent Dye for 100 Percent Gray Coverage, Teal Forest TL1"
108560,B094D42LRN,"[best educational toys for 2 year old, educational toys for 2 year old boys]",2,"Montessori Toys for 1 to 3-Year-Old Boys Girls Toddlers, Wooden Sorting & Stacking Toys for Toddlers and Kids Preschool, Educational Toys, Color Recognition Stacker Shape Sorter, Learning Puzzles Gift"
108621,B094QX351C,"[# 2 pencils not sharpened, #2 pencils with erasers sharpened not soft]",2,Neon Pencils for Kids HB Wood Pencil with Eraser Fluorescent Colored Wood Pencils Colorful Round Pencils Writing Drawing Pencils School Student Reward Supplies (60)
108630,B094VK267S,"[safety 1st stay clean humidifier, humidifier without filter for large room]",2,"Humidifiers for Home - 4.5L Cool Mist Ultrasonic Humidifier with 26db Quiet Operation and Auto Shut Off, 75H Work Time Humidifiers for Bedroom , Baby Room, Office (Gray)"
108639,B094X9T138,"[cat urine enzyme cleaner for carpet cleaner, commercial steam cleaner detergent]",2,Zeiff Pro-Grade Multi-Purpose Enzyme Cleaner - Powerful Cleaning & Odor Eliminating Formula For Professional & Home Surfaces - 1 Gallon - Lavender Breeze
108675,B095388CLF,"[women sexy toyes bluetooth, lush 2 toys for women adult]",2,"Ben Wa Balls,Kegel Balls for Women with App Remote Control,Doctor Recommended Kegel Balls Exercise Product for Pelvic Floor Tightening and Bladder Control Strengthening"
108703,B095BQF1HB,"[pour wine without removing cork, wine opener without removing cork]",2,Coravin Timeless Three SL Wine Bottle Opener and Preservation System
108704,B095BRZYP2,"[pour wine without removing cork, wine opener without removing cork]",2,Coravin Timeless Three + Wine Preservation System


In [48]:
len(df_res)


9937

In [49]:
mitosheet.sheet(df_res)

In [50]:
df_res.to_csv("../data/candidates.csv", index=False)

In [51]:
product_ids = [
    
    
"60256656",
"60256672",
"62352156",
"62404520",
"316425168",
"316460834",
"385345623",
"394551907",
"399557288",
"448462656",
"451224957",
"545396344",
"545795664",
"615991424",
"671530771",
"735213682",
"800722299",
"802412858",
"803736800",
"823085783",
"979017300",
"997799374",
"1421595796",
"1426310366",
"1492675377",
"1628602570",
"1633226247",
"1633535835",
"1641520922",
"1936976021",
"1943200106",
"1951025083",
"B00002ND41",
"B00004W499",
"B000052XFC",
"B000052YOX",
"B00005MEGV",
"B00005O6B7",
"B00005YX2Z",
"B000066SY1",
"B0000D8EJV",
"B0000DIX7S",
"B06X17LJF9",
"B07V3QS2CY",
"B07VC383LJ",
"B07XVP6N3H",
"B07ZFKKTK9",
"B089ND563V",
"B08CRR557D",
"B08QFTSLJD",
"B08QHPJG1P",
"B08Y74HSSH",
"B08Y8NVW6Y",
"B096GR9NX9"]

In [55]:
df_candidates = df_res [ df_res['product_id'].isin(product_ids) ]
df_candidates

,product_id,queries,num_queries,title
2110,1421595796,"[pokemon book pokedex, pokemon illustration book]",2,Let's Find Pokémon! Special Complete Edition (2nd edition)
2167,1426310366,"[planets books for kids 3-5, national geographic little kids first big book of space, children space book]",3,National Geographic Readers: Planets
2470,1492675377,"[gifts under 10 dollars, gifts under 15 dollars, small book of jokes for kids]",3,Dad Jokes: Over 600 of the Best (Worst) Jokes Around and Perfect Holiday Stocking Stuffer (World's Best Dad Jokes Collection)
3080,1628602570,"[soul food cookbook for beginners, chicken soup for nutrient]",2,Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy
3113,1633226247,"[gifts for 1 year old girl, birthday books for 1 year old]",2,"ABC for Me: ABC What Can She Be?: Girls can be anything they want to be, from A to Z (ABC for Me, 5)"
3115,1633535835,"[14 year old girl gifts, 20 best history books]",2,"The Book of Awesome Women: Boundary Breakers, Freedom Fighters, Sheroes and Female Firsts (Teenage girl gift)"
3153,1641520922,"[autism counseling work book, age 4 boy conflict book]",2,Anger Management Workbook for Kids: 50 Fun Activities to Help Children Stay Calm and Make Better Choices When They Feel Mad (Health and Wellness Workbooks for Kids)
3543,1936976021,"[gifts for 18 year old boys, valentine gifts for teenage girl]",2,How Not to Be a D*ck: An Everyday Etiquette Guide
3595,1943200106,"[birthday gifts for girlfriend, cute gifts for girlfriends]",2,"I Wrote a Book About You — A fun, fill-in-the-blank book."
3662,1951025083,"[stocking stuffers for 11 year old girls, funny stocking stuffers]",2,The Try Not to Laugh Challenge - Would Your Rather? - EWW Edition


[pokemon book pokedex, pokemon illustration book]	2	Let's Find Pokémon! Special Complete Edition (2nd edition)

[soul food cookbook for beginners, chicken soup for nutrient]	2	Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy

[stocking stuffers for 11 year old girls, funny stocking stuffers]	2	The Try Not to Laugh Challenge - Would Your Rather? - EWW Edition


[whole house fan, large whole house humidifiers]	2	AIRCARE Digital Whole-House Console-Style Evaporative Humidifier (Copper Night)

[life without plastic toothbrush, eco friendly toothbrushes medium bristles]	2	Biodegradable Bamboo Toothbrush, Natural Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable Travel Toothbrushes Wooden toothbrushes, 6 Pack

Examples:   
Input:   
Title: Let's Find Pokémon! Special Complete Edition (2nd edition)   
Output:    
pokemon book pokedex   
pokemon illustration book   
Input:   
Title: Keto Comfort Foods: Family Favorite Recipes Made Low-Carb and Healthy   
Output:   
soul food cookbook for beginners   
chicken soup for nutrient 
Input:   
Title: The Try Not to Laugh Challenge - Would Your Rather? - EWW Edition
Output:   
stocking stuffers for 11 year old girls
funny stocking stuffers
Input:  
Title: AIRCARE Digital Whole-House Console-Style Evaporative Humidifier (Copper Night)
Output:  
whole house fan
large whole house humidifiers
Input:  
Title: Biodegradable Bamboo Toothbrush, Natural Charcoal toothbrushes Soft Bristle Toothbrush Eco-Friendly Sustainable Toothbrush BPA Free Organic Compostable Travel Toothbrushes Wooden toothbrushes, 6 Pack
Output:  
life without plastic toothbrush
eco friendly toothbrushes medium bristles